#### Objective: Obtain out of fold predictions on the entire training set using cross validation and then using a mean average precision IoU metric, that closely resembles the competition metric, to improve validation

##### reference1: https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein
##### reference2: https://www.kaggle.com/cchadha/mean-average-precision-iou-on-cnn-oof-preds

In [1]:
import numpy as np
import pandas as pd

#### Prepare out of fold training predictions for implementation of MAP IoU matching competition evaluation description

Load submission file which is made by myweight.h5

In [2]:
oof_preds0 = pd.read_csv('./root_dir/submission.csv')

In [3]:
oof_preds0.head()

,patientId,PredictionString
0,0092d9c5-26b6-4e66-b196-49b2224ab8d1,0.96 226.0 592.0 200.0 236.0
1,2eeca9b5-d14f-46c6-ac7e-f676ee83e6be,NaN
2,021082a9-a734-42a8-bbe2-233e19a59103,NaN
3,153d2c23-546a-4fc4-93bc-7873e24d50fa,0.98 218.0 446.0 180.0 182.0
4,270d287b-5f32-469d-9f88-f4a87f27ef47,0.98 678.0 406.0 152.0 144.0


Read true ground bounding box file

In [4]:
df = pd.read_csv('./input/testing_file_20181205.csv')
df.head()

,patientId,x,y,width,height,Target
0,000924cf-0f8d-42bd-9158-1af53881a557,NaN,NaN,NaN,NaN,0
1,000db696-cf54-4385-b10b-6b16fbb3f985,316.0,318.0,170.0,478.0,1
2,000db696-cf54-4385-b10b-6b16fbb3f985,660.0,375.0,146.0,402.0,1
3,000fe35a-2649-43d4-b027-e67796d412e0,570.0,282.0,269.0,409.0,1
4,000fe35a-2649-43d4-b027-e67796d412e0,83.0,227.0,296.0,438.0,1


Parse bounding box labels into correct format for Mean Average Precision IoU metric

In [5]:
# make a bbox_target column using x, y, width and height.
df['bbox_target'] = (df['x'].astype(str) +
                    ' ' + 
                    df['y'].astype(str) +
                    ' ' +
                    df['width'].astype(str) +
                    ' ' +
                    df['height'].astype(str))

In [6]:
# split by space, ',' Is printed as the default.',' between two values
df.loc[:, 'bbox_target'] = df.loc[:, 'bbox_target'].map(lambda x: x.split(' '))
df.head()

,patientId,x,y,width,height,Target,bbox_target
0,000924cf-0f8d-42bd-9158-1af53881a557,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
1,000db696-cf54-4385-b10b-6b16fbb3f985,316.0,318.0,170.0,478.0,1,"[316.0, 318.0, 170.0, 478.0]"
2,000db696-cf54-4385-b10b-6b16fbb3f985,660.0,375.0,146.0,402.0,1,"[660.0, 375.0, 146.0, 402.0]"
3,000fe35a-2649-43d4-b027-e67796d412e0,570.0,282.0,269.0,409.0,1,"[570.0, 282.0, 269.0, 409.0]"
4,000fe35a-2649-43d4-b027-e67796d412e0,83.0,227.0,296.0,438.0,1,"[83.0, 227.0, 296.0, 438.0]"


In [7]:
# 'patientId'를 기준으로 정렬 후, 동일 patientId가 있을 경우 ( if there is same patientId, sorting by 'patientId')
# bbox_target값 sum 하는데 string이라 옆으로 합쳐진다.
df = df.groupby(['patientId'], as_index = False)['bbox_target'].agg('sum')    
df.head()

,patientId,bbox_target
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]"
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296...."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376...."
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]"


Merge labels and oof preds

In [8]:
#merge by 'patientId', delete if there is no same id! 동일한 아이디가 없으면 삭제하기!
df = df.merge(oof_preds0, on = 'patientId', how = 'right')
df.head()

,patientId,bbox_target,PredictionString
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",NaN
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.97 610.0 406.0 196.0 348.0 0.96 288.0 300.0...
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.99 94.0 256.0 278.0 358.0 0.99 548.0 300.0 ...
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.97 596.0 356.0 298.0 400.0
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",NaN


In [9]:
#3개의 파일을 merge 해서 PredictionString(x, y, default)로 뜸
df = df.fillna('') # none -> " "

In [10]:
df.head()

,patientId,bbox_target,PredictionString
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.97 610.0 406.0 196.0 348.0 0.96 288.0 300.0...
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.99 94.0 256.0 278.0 358.0 0.99 548.0 300.0 ...
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.97 596.0 356.0 298.0 400.0
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",


Parse oof preds for MAP IoU

In [11]:
#필요없는 열 제거, Remove unnecessary column
#df = df.drop(['PredictionString','PredictionString_x', 'PredictionString_y'], axis=1)
df.loc[:, 'bbox_pred'] = (df.loc[:, 'PredictionString'])
df = df.drop(['PredictionString'], axis=1)
df.head(20)

,patientId,bbox_target,bbox_pred
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.97 610.0 406.0 196.0 348.0 0.96 288.0 300.0...
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.99 94.0 256.0 278.0 358.0 0.99 548.0 300.0 ...
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.97 596.0 356.0 298.0 400.0
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",
5,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",
6,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.98 530.0 200.0 306.0 540.0 0.98 124.0 204.0...
7,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",
8,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",
9,00293de0-a530-41dc-9621-0b3def01d06d,"[nan, nan, nan, nan]",0.96 166.0 606.0 240.0 194.0


Stripping whitespace from PredictionString column

In [12]:
# bbox_pred 값을 문자열로 변환 후 문자열의 양끝에서 공백, 탭, 개행문자 등을 제거
df = df.fillna('') # none -> " "로 처리
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].str.strip()
df.tail(20)

,patientId,bbox_target,bbox_pred
980,c145df8e-9631-468d-af7f-5690c11c2c88,"[601.0, 265.0, 173.0, 383.0, 173.0, 370.0, 178...",0.99 156.0 360.0 200.0 328.0
981,c14d9ceb-019f-45f6-9299-281b58de57df,"[578.0, 447.0, 157.0, 258.0, 239.0, 268.0, 172...",0.97 186.0 356.0 226.0 432.0
982,c1592aae-c80d-4794-ab28-463905558534,"[nan, nan, nan, nan]",
983,c1628c47-5ba3-42dd-8df3-7ad3abd57ad0,"[622.0, 356.0, 149.0, 128.0]",0.96 600.0 210.0 218.0 328.0
984,c164b17b-aff8-484f-9d9e-dde2932d8df9,"[nan, nan, nan, nan]",
985,c1718678-44af-407f-829a-fc65bc854094,"[nan, nan, nan, nan]",0.96 132.0 436.0 224.0 144.0
986,c18d1138-ba74-4af5-af21-bdd4d2c96bb5,"[646.0, 386.0, 141.0, 206.0, 295.0, 473.0, 133...",0.97 266.0 466.0 162.0 190.0
987,c196ce23-f37c-4ab3-a9ce-ea8ede90e09c,"[nan, nan, nan, nan]",
988,c19b8a3b-ab4e-4a73-8e13-ec0a84b6b6c7,"[nan, nan, nan, nan]",
989,c1c3ec5d-20ba-42f7-91f9-48032d97ffc9,"[nan, nan, nan, nan]",


In [13]:
# 공백을 기준으로 나눔, ','는 기본으로 찍힘
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].map(lambda x: x.split(' '))
df.head()

,patientId,bbox_target,bbox_pred
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.97, 610.0, 406.0, 196.0, 348.0, 0.96, 288.0..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.99, 94.0, 256.0, 278.0, 358.0, 0.99, 548.0,..."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....","[0.97, 596.0, 356.0, 298.0, 400.0]"
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",[]


In [14]:
def parse_scores(x):
    if len(x)!=1:
        scores = [x[k] for k in range(0,len(x),5)]
        for score in range(len(scores)):
            scores[score] = float(scores[score])
        return np.asarray(scores)
    #score 추출

In [15]:
df.loc[:, 'bbox_scores'] = df.loc[:, 'bbox_pred'].map(parse_scores)

In [16]:
df.tail()

,patientId,bbox_target,bbox_pred,bbox_scores
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.96, 310.0, 480.0, 192.0, 264.0]",[0.96]
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.98, 186.0, 316.0, 242.0, 366.0]",[0.98]
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",[],None
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.97, 224.0, 514.0, 214.0, 236.0]",[0.97]


In [17]:
#extracting bounding box location
def parse_bbox(x):
    if len(x)!=1:
        bbox = [(x[k]) for k in range(0,len(x)) if k%5 != 0] #int 삭제
        return np.asarray(bbox).reshape(int(len(bbox)/4),4)

In [18]:
df.loc[:, 'bbox_preds'] = df.loc[:, 'bbox_pred'].map(parse_bbox)

In [19]:
df.tail()

,patientId,bbox_target,bbox_pred,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...","[0.96, 310.0, 480.0, 192.0, 264.0]",[0.96],"[[310.0, 480.0, 192.0, 264.0]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...","[0.98, 186.0, 316.0, 242.0, 366.0]",[0.98],"[[186.0, 316.0, 242.0, 366.0]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None,None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",[],None,None
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...","[0.97, 224.0, 514.0, 214.0, 236.0]",[0.97],"[[224.0, 514.0, 214.0, 236.0]]"


In [20]:
#bbox_pred 추출 후 열 제거, removing column 'bbox_pred'
df = df.drop(['bbox_pred'], axis=1)

In [21]:
df.tail()

,patientId,bbox_target,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...",[0.96],"[[310.0, 480.0, 192.0, 264.0]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...",[0.98],"[[186.0, 316.0, 242.0, 366.0]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",None,None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",None,None
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...",[0.97],"[[224.0, 514.0, 214.0, 236.0]]"


Edit NaN or None values to empty numpy arrays to fit MAP IoU metric implementation

In [22]:
df.loc[df['bbox_scores'].isnull(),['bbox_scores']] = df.loc[df['bbox_scores'].isnull(),'bbox_scores'].apply(lambda x: np.asarray([]))

In [23]:
#remove null in bbox_scores and print 
df.tail()

,patientId,bbox_target,bbox_scores,bbox_preds
995,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,"[666.0, 418.0, 186.0, 223.0, 316.0, 504.0, 179...",[0.96],"[[310.0, 480.0, 192.0, 264.0]]"
996,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,"[609.0, 464.0, 240.0, 284.0, 185.0, 298.0, 228...",[0.98],"[[186.0, 316.0, 242.0, 366.0]]"
997,c1edf42b-5958-47ff-a1e7-4f23d99583ba,"[nan, nan, nan, nan]",[],None
998,c1f6b555-2eb1-4231-98f6-50a963976431,"[nan, nan, nan, nan]",[],None
999,c1f7889a-9ea9-4acb-b64c-b737c929599a,"[570.0, 393.0, 261.0, 345.0, 233.0, 424.0, 201...",[0.97],"[[224.0, 514.0, 214.0, 236.0]]"


In [24]:
df.loc[df['bbox_preds'].isnull(),['bbox_preds']] = df.loc[df['bbox_preds'].isnull(),'bbox_preds'].apply(lambda x: np.asarray([]))

In [25]:
#remove null in bbox_pred and print
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.97, 0.96]","[[610.0, 406.0, 196.0, 348.0], [288.0, 300.0, ..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.99, 0.99]","[[94.0, 256.0, 278.0, 358.0], [548.0, 300.0, 2..."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",[0.97],"[[596.0, 356.0, 298.0, 400.0]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",[],[]


In [26]:
def parse_target_str(x):
    if x[0] != 'nan':
        bbox = np.asarray([int(float(x[k])) for k in range(0,len(x))])
        return bbox.reshape(int(len(bbox)/4),4)

In [27]:
#bbox_target값 4개씩 묶어서 재 배열
df.loc[:,'bbox_target'] = df.loc[:,'bbox_target'].map(parse_target_str)
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,None,[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[[316, 318, 170, 478], [660, 375, 146, 402]]","[0.97, 0.96]","[[610.0, 406.0, 196.0, 348.0], [288.0, 300.0, ..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[[570, 282, 269, 409], [83, 227, 296, 438]]","[0.99, 0.99]","[[94.0, 256.0, 278.0, 358.0], [548.0, 300.0, 2..."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[[66, 160, 373, 608], [552, 164, 376, 676]]",[0.97],"[[596.0, 356.0, 298.0, 400.0]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,None,[],[]


In [28]:
# remove null in bbox_target and print
df.loc[df['bbox_target'].isnull(),['bbox_target']] = df.loc[df['bbox_target'].isnull(),'bbox_target'].apply(lambda x: np.asarray([]))

In [29]:
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,000924cf-0f8d-42bd-9158-1af53881a557,[],[],[]
1,000db696-cf54-4385-b10b-6b16fbb3f985,"[[316, 318, 170, 478], [660, 375, 146, 402]]","[0.97, 0.96]","[[610.0, 406.0, 196.0, 348.0], [288.0, 300.0, ..."
2,000fe35a-2649-43d4-b027-e67796d412e0,"[[570, 282, 269, 409], [83, 227, 296, 438]]","[0.99, 0.99]","[[94.0, 256.0, 278.0, 358.0], [548.0, 300.0, 2..."
3,001031d9-f904-4a23-b3e5-2c088acd19c6,"[[66, 160, 373, 608], [552, 164, 376, 676]]",[0.97],"[[596.0, 356.0, 298.0, 400.0]]"
4,0010f549-b242-4e94-87a8-57d79de215fc,[],[],[]


### measure IoU and mAP

In [30]:
# helper function to calculate IoU
def iou(box1, box2):
    box1 = box1.astype(np.float)
    box2 = box2.astype(np.float)
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [31]:
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    if len(boxes_true) == 0 and len(boxes_pred) == 0:
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :]

        map_total = 0 # this value is for kaggle mAP
        map_threshold = []

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched:
                    fn += 1 # bt has no match, count as FN

            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp)
            map_threshold.append(m) # AP per each thresholds
            map_total += m
    
        map_total=map_total / len(thresholds)
        map_threshold.append(map_total) # for total mAP
    
    return map_threshold

In [33]:
#map_iou test, print upper 10 mAP values
for row in range(10):
    print(map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]))

None
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.0, 0.7916666666666666]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125]
None
0
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.875]
None
None
0
None
[1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.4583333333333334]
None
None
None
None
None
[1.0, 1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.5]
None
None


In [34]:
#measure AP non이 아닐때만 map_iou 계산
map_scores = [
    x for x in [map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]) for row in range(len(df))] if x is not None]

In [35]:
map_scores # check values

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.0, 0.7916666666666666],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125],
 0,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.875],
 0,
 [1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.0,
  0.4583333333333334],
 [1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.0,
  0.0,
  0.5],
 0,
 [1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.0,
  0.0,
  0.5],
 0,
 0,
 [0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.25],
 0,
 0,
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.7499999999999999],
 0,
 0,
 0,
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 0,
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [43]:
# if mAP is 0, then change the 0 to [0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(map_scores)):
    if map_scores[i] == 0 :
        del map_scores[i]
        map_scores.insert(i,[0, 0, 0, 0, 0, 0, 0, 0, 0])

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0

In [45]:
# reverse list, [i][j] -> [j][i]
new_list = list(map(list, zip(*map_scores)))

In [48]:
for i in range(len(new_list)):
    print(np.mean(new_list[i])) #model mAP of 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, total threshold

0.28125
0.25291053921568624
0.2193627450980392
0.18275122549019607
0.1488970588235294
0.10830269607843135
0.07337622549019607
0.04764093137254902
0.16431142769607843
